# 02 Correct Network Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [ ]:
root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_02')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-27 14:37:58, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 14:37:58, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 00 Scenario

In [61]:
version_00_pickle_file_name = os.path.join(input_dir, 'working_scenario_00.pickle')
v_00_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [62]:
v_00_scenario.road_net.links_df.shape

(1634769, 30)

In [63]:
v_01_scenario = v_00_scenario

# Add centroid and centroid connectors
before applying project cards

In [64]:
v_01_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = v_01_scenario.road_net,
    parameters = parameters)

2020-10-27 16:28:56, INFO: Adding centroid and centroid connector to standard network
2020-10-27 16:28:56, INFO: Adding centroid and centroid connector to standard network
2020-10-27 16:28:56, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:28:56, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:29:10, INFO: Finished adding centroid and centroid connectors
2020-10-27 16:29:10, INFO: Finished adding centroid and centroid connectors


In [65]:
print(v_01_scenario.road_net.links_df.shape)
print(v_01_scenario.road_net.nodes_df.shape)
print(v_01_scenario.road_net.shapes_df.shape)

(2221035, 30)
(688952, 11)
(1163103, 7)


In [68]:
# lanes for connectors

v_01_scenario.road_net = mtc.determine_number_of_lanes(
    roadway_network = v_01_scenario.road_net,
    parameters = parameters,
    network_variable = 'lanes'
)

v_01_scenario.road_net.links_df.lanes.value_counts()

2020-10-27 16:30:58, INFO: Determining number of lanes
2020-10-27 16:30:58, INFO: Determining number of lanes
2020-10-27 16:30:58, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:30:58, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:37:34, INFO: Finished determining number of lanes using variable: lanes
2020-10-27 16:37:34, INFO: Finished determining number of lanes using variable: lanes


1    2072663
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: lanes, dtype: int64

## Create Scenario and Apply Project Cards

In [ ]:
# TODO: replace with tag fetch once debug is done
name_list = glob.glob(card_dir + '/*.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_i80_wb_hov_ala.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_bus_only_ggt_on_us101.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_attributes_exclude_trucks_sr85.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_managed_lane_i80_ramp_to_bay_bridge.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_attributes_exclude_trucks_i580.yml')
name_list.remove('/Users/wsp/Documents/GitHub/travel-model-two-networks/project_cards/year_2015_bus_only_transbay_terminal_ramps.yml')
card_list = [] 
for name in name_list:
    card = ProjectCard.read(os.path.join(card_dir, name), validate = False)
    card_list.append(card)

In [69]:
for card in card_list:
    v_01_scenario.apply_project(card)

2020-10-27 16:37:54, INFO: Applying Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
2020-10-27 16:37:54, INFO: Applying Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
2020-10-27 16:37:54, INFO: Applying Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
2020-10-27 16:37:54, INFO: Applying Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
2020-10-27 16:37:54, INFO: Applying Project to Roadway Network: Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
2020-10-27 16:37:54, INFO: Applying Project to Roadway Network: Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [70]:
v_01_scenario.applied_projects

['Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma']

In [71]:
v_01_scenario.road_net.links_df.shape

(2221035, 36)

In [72]:
v_01_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry', 'ft', 'useclass',
       'assignable', 'transit', 'managed', 'ML_lanes', 'segment_id',
       'ML_useclass', 'ML_access', 'ML_egress'],
      dtype='object')

In [73]:
v_01_scenario.road_net.links_df.managed.value_counts()

0    2221005
1         30
Name: managed, dtype: int64

## Make Travel Model Networks

In [74]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-10-27 16:44:40, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:44:40, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [75]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2221035, 36)
(688952, 11)
(1163103, 7)


In [76]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2020-10-27 16:45:32, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-27 16:45:32, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
(2221035, 36)
2020-10-27 16:45:32, INFO: Creating managed lane network.
2020-10-27 16:45:32, INFO: Creating managed lane network.
2020-10-27 16:45:32, INFO: Creating network with duplicated managed lanes
2020-10-27 16:45:32, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(2221125, 36)
2020-10-27 16:46:27, INFO: Calculating Facility Type
2020-10-27 16:46:27, INFO: Calculating Facility Type
2020-10-27 16:46:27, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:46:27, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:46:27, INFO: Variable 'ft' updated for some links. Returning without overwriting for those links. Calculating for other links
2020-10-27 16:46:27, INFO: Variable 'ft' updated for some links. Returning without overwriting for those links. Calculating for other links
2020-10-27 16:49:31, INFO: Finished calculating roadway class variable: ft
2020-10-27 16:49:31, INFO: Finished calculating roadway class variable: ft
2020-10-27 16:49:32, INFO: Determining assignable
2020-10-27 16:49:32, INFO: Determining assignable
2020-10-27 16:49:32, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-10-27 16:49:32, INFO: Lasso base directory set as: Z:/Data/Users/S

In [77]:
model_net.links_df.lanes_AM.value_counts()

1     2072753
2      104198
3       22434
4       14124
5        6504
6         929
7         134
88         30
8          18
9           1
Name: lanes_AM, dtype: int64

In [78]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2221125, 53)
(688983, 13)
(1163193, 8)


## Write to Disk

## As shapefile

In [80]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2020-10-27 16:56:12, INFO: Writing Network as Shapefile
2020-10-27 16:56:12, INFO: Writing Network as Shapefile
2020-10-27 16:56:12, INFO: Renaming DBF Node Variables
2020-10-27 16:56:12, INFO: Renaming DBF Node Variables
2020-10-27 16:56:12, INFO: Renaming variables so that they are DBF-safe
2020-10-27 16:56:12, INFO: Renaming variables so that they are DBF-safe
2020-10-27 16:56:25, INFO: Renaming DBF Link Variables
2020-10-27 16:56:25, INFO: Renaming DBF Link Variables
2020-10-27 16:56:25, INFO: Renaming variables so that they are DBF-safe
2020-10-27 16:56:25, INFO: Renaming variables so that they are DBF-safe
2020-10-27 16:56:33, INFO: Writing Node Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks\data\processed\version_02\nodes.shp
2020-10-27 16:56:33, INFO: Writing Node Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks\data\processed\version_02\nodes.shp
2020-10-27 16:57:25, INFO: Writing Link Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-mod

## As fixed width (for CUBE)

In [ ]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

In [ ]:
from subprocess import Popen

os.chdir(output_dir)
p = Popen("make_cube.bat")
p.communicate()

## As pickle

In [ ]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(working_scenario_filename, 'wb'))